# IP-Geoloc scrap

Scrap from https://tools.keycdn.com/geo

In [1]:
# libraries

import requests
from bs4 import BeautifulSoup as bs
import pymongo 
import re

In [2]:
# reading from mongo

client=pymongo.MongoClient()
db=client.geo_attack

geo=list(db.geo.find())

In [3]:
# predator ip list

list_ip=[e['ip_pred'] for e in geo]

In [4]:
# get

res=requests.get('https://tools.keycdn.com/geo?host={}'.format(list_ip[0])).content
print (res)
print (type(res))

b'\n<!DOCTYPE html>\n<html lang="en" prefix="og: http://ogp.me/ns#">\n    <head>\n        <meta charset="utf-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n        <meta name="version" content="7d4b0428889afce8783b1678d81467cc72dba0e1">\n\n        <title>IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools</title>\n        <meta name="description" content="What is my IP address? Lookup an IP address or hostname to get the geographical location.">\n\n        <meta name="twitter:card" content="summary_large_image">\n        <meta name="twitter:site" content="@KeyCDN">\n        <meta name="twitter:creator" content="@KeyCDN">\n        <meta property="og:url" content="https://tools.keycdn.com/geo?host=45.33.246.179">\n        <meta property="og:type" content="website">\n        <meta property="og:title" content="IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools">\n        <meta property=

In [5]:
# bs4

soup=bs(res, features='lxml')
print (soup)
print (type(soup))

<!DOCTYPE html>
<html lang="en" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="7d4b0428889afce8783b1678d81467cc72dba0e1" name="version"/>
<title>IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools</title>
<meta content="What is my IP address? Lookup an IP address or hostname to get the geographical location." name="description"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@KeyCDN" name="twitter:site"/>
<meta content="@KeyCDN" name="twitter:creator"/>
<meta content="https://tools.keycdn.com/geo?host=45.33.246.179" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools" property="og:title"/>
<meta content="What is my IP address? Lookup an IP address or hostname to get the geographical location." property="og:descript

In [6]:
# find element

elem=soup.find_all('dd', {'class': 'col-8 text-monospace'})
print (elem)
print (type(elem))

[<dd class="col-8 text-monospace">United States (US)</dd>, <dd class="col-8 text-monospace">North America (NA)</dd>, <dd class="col-8 text-monospace">37.751 (lat) / -97.822 (long)</dd>, <dd class="col-8 text-monospace">2019-08-13 16:15:51 (America/Chicago)</dd>, <dd class="col-8 text-monospace">45.33.246.179</dd>, <dd class="col-8 text-monospace">45.33.246.179</dd>, <dd class="col-8 text-monospace">HT</dd>, <dd class="col-8 text-monospace">26658</dd>]
<class 'bs4.element.ResultSet'>


In [ ]:
# start scrap


for i in range(0, len(list_ip), 50):

    try:
        res=requests.get('https://tools.keycdn.com/geo?host={}'.format(list_ip[i])).content

        soup=bs(res, features='lxml')

        elem=soup.find_all('dd', {'class': 'col-8 text-monospace'})


        for f in elem:
            F=str(f)
            if 'lat' in F: 
                lat=re.findall('-?\d+.\d+',F.split('/')[0])[0]
                lng=re.findall('-?\d+.\d+',F.split('/')[1])[0]
                
                print ('')
                print ('Ip : {}'.format(list_ip[i]))
                print ({'latitude': lat, 'longitude': lng})
                
    except:
        continue